# BA2plus Team Submission

## 필수 기재내용

### 전체 프로세스 개요:

- 자체제작한, 별도의 크롤링 패키지로 KRX/Naver 에서 가격, 거개량, 유동성, 시총 등의 데이터를 가져옴. 
    - 코드: https://github.com/jaepil-choi/korquanttools
    - 각각의 데이터를 `.pickle` 로 저장해 본 코드에서 써먹음 
    - pickle data, train dataset을 받아올 수 있는 google drive 첨부: https://drive.google.com/drive/folders/1fR9fiogdhdktHuX_LbgBdMxfgHSkFT1y?usp=sharing
- 데이터를 불러와 public 기간 전까지 자르고, 데이터를 조합하여 새로운 변수를 만듦
    - 따라서 look-ahead 없음
    - 새로운 변수: `close t-1`, `close t-3`, `close t-5`, `normalized rdv/adv`
    - 각 세팅은 별도의 `submission_config.py` 모듈로 관리. (뒷부분 첨부)
- 모델에 넣고 돌림 
    - XGB + LGBM base model 
    - step 1, step 2, ... , step 15에 대해 따로 예측함 
    - 시그널 만듦 
- submission 형식에 맞게 변환
    - `submission_util.py` 모듈로 형식에 맞게 변환함. (뒷부분 첨부)

### 코드 실행환경 및 실행방법
- 코드 실행환경
    - python 3.9
    - xgboost, sklearn, lightgbm, tqdm, pandas, numpy 필요
    - prophet, pycaret, catboost, pandas_ta 필요
- 실행방법 (중요)
    - .ipynb만 제출할 수 있다는 대회 제약 때문에 부득이 .py 모듈을 후반부에 첨부. 이 파일들이 있어야 코드가 돌아감. 
    - 코드를 실행하려면 drive 링크의 pickle 파일들을 받아 `/data` 폴더에 넣고, output을 넣을 `/output` 폴더도 만들어줘야 함.
    - 또한, `/data` 폴더 내에 `train.csv`, `train_additional.csv`파일을 넣어야 함.
    - 그리고 노트북과 같은 폴더 안에 `submission_config.py` 와 `submission_config.py`, `sample_submission.csv` 파일이 위치해야 함. 


나머지 과정은 아래 markdown 참고 부탁드립니다.

# Alpha Stretagy by Jaepil

## Basic settings

### Import libraries

In [11]:
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

from tqdm import tqdm

In [13]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.impute import SimpleImputer

#import statsmodels.api as sm
#import statsmodels.formula.api as smf

In [14]:
## custom library

import eda_util as eutil
import submission_config as subconfig
import submission_util as subutil

In [15]:
pd.set_option('display.float_format', lambda x: f'{x:,g}')

In [16]:
BASE_PATH = subconfig.BASE_PATH
DATA_PATH = subconfig.DATA_PATH

OUTPUT_PATH = subconfig.OUTPUT_PATH

### Import data & preprocessing

In [17]:
krx_df = pd.read_csv(subconfig.krx_df_PATH)

In [18]:
krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']

In [19]:
krx_df['date'] = pd.to_datetime(krx_df['date'], format='%Y%m%d')

In [20]:
return_df = pd.read_pickle(subconfig.return_df_PATH)
close_df = pd.read_pickle(subconfig.adjclose_df_PATH)

In [21]:
open_df = pd.read_pickle(subconfig.adjopen_df_PATH)
high_df = pd.read_pickle(subconfig.adjhigh_df_PATH)
low_df = pd.read_pickle(subconfig.adjlow_df_PATH)

In [22]:
## date list

holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

In [23]:
TRAIN_START = pd.to_datetime(subconfig.TRAIN_START, format='%Y-%m-%d')
REALOS_PORTFOLIO_DATE = pd.to_datetime(subconfig.REALOS_PORTFOLIO_DATE, format='%Y-%m-%d')

In [24]:
tradingdays = tradingdays[(tradingdays >= TRAIN_START) & (tradingdays <= REALOS_PORTFOLIO_DATE)]

In [25]:
dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()]

In [26]:
return_df = return_df.loc[tradingdays, :].dropna(axis='columns', how='all')
return_df = return_df.loc[:, dacon_sid_list]

close_df = close_df.loc[tradingdays, :].dropna(axis='columns', how='all')
close_df = close_df.loc[:, dacon_sid_list]

In [27]:
open_df = open_df.loc[tradingdays, :].dropna(axis='columns', how='all')
open_df = open_df.loc[:, dacon_sid_list]

high_df = high_df.loc[tradingdays, :].dropna(axis='columns', how='all')
high_df = high_df.loc[:, dacon_sid_list]

low_df = low_df.loc[tradingdays, :].dropna(axis='columns', how='all')
low_df = low_df.loc[:, dacon_sid_list]

In [28]:
# SIMOS_START = subconfig.SIMOS_START
# simOS_END = subconfig.SIMOS_END

### Import additional data

In [30]:
volume_df = pd.read_pickle(subconfig.volume_df_PATH)
dollarvolume_df = pd.read_pickle(subconfig.dollarvolume_df_PATH)
marketcap_df = pd.read_pickle(subconfig.marketcap_df_PATH)
market_cat_df = pd.read_pickle(DATA_PATH / 'market_cat_df_20140101_20230730.pickle')

In [31]:
volume_df = volume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
volume_df = volume_df.loc[:, dacon_sid_list]

dollarvolume_df = dollarvolume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
dollarvolume_df = dollarvolume_df.loc[:, dacon_sid_list]

marketcap_df = marketcap_df.loc[tradingdays, :].dropna(axis='columns', how='all')
marketcap_df = marketcap_df.loc[:, dacon_sid_list]

In [32]:
# Don't shift data since Insoo's code already makes a shift. 

# volume_df = volume_df.shift(1)
# dollarvolume_df = dollarvolume_df.shift(1)
# marketcap_df = marketcap_df.shift(1)

In [33]:
market_cat_inrange = market_cat_df[market_cat_df['trdDd'].isin(tradingdays)]

In [34]:
KOSPI_sid_list = market_cat_inrange[market_cat_inrange['is_KOSPI'] == True]['ISU_SRT_CD'].unique()
KOSDAQ_sid_list = market_cat_inrange[market_cat_inrange['is_KOSDAQ'] == True]['ISU_SRT_CD'].unique()
KONEX_sid_list = market_cat_inrange[market_cat_inrange['is_KONEX'] == True]['ISU_SRT_CD'].unique()

### Parameters

In [35]:
REALOS_PORTFOLIO_DATE = subconfig.REALOS_PORTFOLIO_DATE

RDVADV_WINDOW = subconfig.WINDOWS['rdvadv'] # 20

### normalized RDV/ADV signal

In [36]:
adv_df = dollarvolume_df.rolling(RDVADV_WINDOW, ).mean().dropna(axis='rows', how='all')

분모: average RDV/ADV ratio


In [37]:
avg_adv_s = adv_df.mean(axis='columns')
avg_rdv_s = dollarvolume_df.iloc[RDVADV_WINDOW:, :].mean(axis='columns')

In [38]:
avg_rdvadv_s = avg_rdv_s / avg_adv_s

분자: individual RDV/ADV ratio 

In [39]:
ii_rdvadv_df = dollarvolume_df.iloc[RDVADV_WINDOW:, :] / adv_df

In [40]:
normalized_rdvadv_signal_df = ii_rdvadv_df.divide(avg_rdvadv_s, axis='rows')
normalized_rdvadv_signal_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-29,0.415442,0.240279,0.405709,0.726369,0.475821,0.907576,0.4201,1.10324,0.539159,0.755196,...,1.81974,0.798739,0.631614,0.340136,14.3229,0.996211,0.600184,NaN,0.369902,0.630198
2021-06-30,0.314977,0.212304,0.465795,0.931287,0.470661,0.514831,0.648498,1.99712,0.354948,0.501368,...,4.39418,8.5227,0.745765,0.319291,7.08257,0.416926,1.40616,NaN,0.291685,0.618186
2021-07-01,0.399208,0.295054,0.570299,0.366269,0.496024,0.584884,1.26369,3.66506,0.356092,0.701445,...,1.03469,1.22015,0.426687,0.971435,1.91106,0.786424,0.705502,NaN,0.495063,0.955606
2021-07-02,1.02063,0.146543,0.639072,0.410245,0.842014,0.260692,1.97162,1.5627,0.724157,0.519005,...,2.03734,0.881352,0.244086,2.64395,0.377858,0.422299,0.421016,NaN,1.13061,0.666445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-24,0.288286,0.911344,1.09077,0.659324,0.56127,0.839701,0.945123,0.375129,0.659149,0.608006,...,0.808454,0.993728,0.444944,1.55736,0.426958,0.929188,0.628311,0.124676,0.649393,0.498613
2023-07-25,0.36685,0.846063,0.898058,1.06346,0.685844,0.617804,0.719524,0.615932,0.496475,0.362459,...,1.02346,1.02446,0.941129,1.2274,7.15383,0.858493,1.16309,0.106905,0.613637,0.343585
2023-07-26,0.255151,1.13615,0.632257,1.09468,0.862278,0.786008,0.910452,0.336899,0.533319,0.558765,...,0.929511,0.958187,0.480403,1.24321,0.582422,1.01109,0.634947,0.0964569,0.485976,0.476295


In [41]:
MODEL_TRAIN_START = pd.to_datetime('2021-06-29', format='%Y-%m-%d')

## Alphas

### Integrating my data with Insoo's code

In [42]:
# Your function to calculate SMAPE
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [43]:
results_df = pd.DataFrame(columns=['code'] + ['return_day_' + str(i) for i in range(1, 16)])

preds_df_fin_xgb = pd.DataFrame()
smapes_df_fin_xgb = pd.DataFrame()

preds_df_fin_lgbm = pd.DataFrame()
smapes_df_fin_lgbm = pd.DataFrame()

preds_df_fin_catboost = pd.DataFrame()
smapes_df_fin_catboost = pd.DataFrame()

아래 코드는 Ryzen 5 5600X 6 Core (CPU 12) 로 돌렸을 때 

40분 가량 걸림. 

Windows에서 GPU 연산은 활용하기 어려움. 

- XGB: conda는 지원안함, Windows는 version conflict 남
- LGBM: Linux만 지원

In [ ]:
# Iterate over each unique stock
for code in tqdm(dacon_sid_list):
    
    # Filter by stock code
    # Note: All prices are adjusted
    # TODO: Add normalized rdvadv signal to the columns

    train_close = pd.DataFrame(
        data={
            'open': open_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'high': high_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'low': low_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'close': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'close_t-1': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(1),
            'close_t-3': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(3),
            'close_t-5': close_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code].shift(5),
            'dollarvolume': dollarvolume_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'marketcap': marketcap_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
            'norm_rdvadv': normalized_rdvadv_signal_df.loc[MODEL_TRAIN_START:REALOS_PORTFOLIO_DATE, code],
        }
        )
    train_close = train_close.iloc[5:, :] # nan 있는 1st row 제거 

    # Store original data for reference
    original_data = train_close.copy()

    # Create return columns for each day
    returns = []
    smapes_xgb = []
    smapes_lgbm = []
    smapes_catboost = []

    preds_df_xgb = pd.DataFrame()
    preds_df_lgbm = pd.DataFrame()
    preds_df_catboost = pd.DataFrame()
    
    # For each day from 1 to 15
    for day in range(1, 16):
        # Scale data
        X = train_close[:]
        y = train_close['close']
        
        scaler = MinMaxScaler(feature_range=(-1, 1))
        data_scaled = scaler.fit_transform(X)
        data_scaled2 = y
        
        X_train = data_scaled[:-day]
        y_train = data_scaled2[day:]
        X_test = data_scaled[-day]
        
        X_train = X_train[:int(len(X_train) * 0.9)]
        X_val = X_train[int(len(X_train) * 0.9):]
        y_train = y_train[:int(len(y_train) * 0.9)]
        y_val = y_train[int(len(y_train) * 0.9):] 

        # Train XGBoost
        xgb_model = XGBRegressor()
        xgb_model.fit(X_train, y_train)
        vals_xgb = xgb_model.predict(X_val)
        smapes_xgb.append(smape(y_val, vals_xgb))
        
        preds_xgb = xgb_model.predict([data_scaled[-day]])
        preds_df_xgb = pd.concat([preds_df_xgb, pd.DataFrame(preds_xgb)], axis = 0)

        # Train LightGBM
        lgbm_model = LGBMRegressor()
        lgbm_model.fit(X_train, y_train)
        vals_lgbm = lgbm_model.predict(X_val)
        smapes_lgbm.append(smape(y_val, vals_lgbm))
        
        preds_lgbm = lgbm_model.predict([data_scaled[-day]])
        preds_df_lgbm = pd.concat([preds_df_lgbm, pd.DataFrame(preds_lgbm)], axis = 0)
    
    smapes_df_xgb = pd.DataFrame(smapes_xgb)
    smapes_df_lgbm = pd.DataFrame(smapes_lgbm)
    smapes_df_catboost = pd.DataFrame(smapes_catboost)

    preds_df_fin_xgb = pd.concat([preds_df_fin_xgb, preds_df_xgb], axis = 1)
    smapes_df_fin_xgb = pd.concat([smapes_df_fin_xgb, smapes_df_xgb], axis = 1)

    preds_df_fin_lgbm = pd.concat([preds_df_fin_lgbm, preds_df_lgbm], axis = 1)
    smapes_df_fin_lgbm = pd.concat([smapes_df_fin_lgbm, smapes_df_lgbm], axis = 1)

In [45]:
# smapes_df_xgb.to_pickle(OUTPUT_PATH / 'smapes_df_xgb.pickle')
# smapes_df_lgbm.to_pickle(OUTPUT_PATH / 'smapes_df_lgbm.pickle')
# smapes_df_catboost.to_pickle(OUTPUT_PATH / 'smapes_df_catboost.pickle')

# preds_df_fin_xgb.to_pickle(OUTPUT_PATH / 'preds_df_fin_xgb.pickle')
# smapes_df_fin_xgb.to_pickle(OUTPUT_PATH / 'smapes_df_fin_xgb.pickle')

# preds_df_fin_lgbm.to_pickle(OUTPUT_PATH / 'preds_df_fin_lgbm.pickle')
# smapes_df_fin_lgbm.to_pickle(OUTPUT_PATH / 'smapes_df_fin_lgbm.pickle')

In [ ]:
smapes_df_fin_xgb.shape

In [35]:
smapes_df_fin_lgbm.shape

(15, 2000)

In [36]:
final = np.zeros((len(smapes_df_fin_xgb), len(smapes_df_fin_xgb.columns)))

for i in range(0, len(smapes_df_fin_xgb.columns)):
    for j in range(0, len(smapes_df_fin_xgb)):
        weights = [1 / smapes_df_fin_xgb.iloc[j:j+1, i].values[0],
                   
                   1 / smapes_df_fin_lgbm.iloc[j:j+1, i].values[0]]
        

        weights /= np.sum(weights) 
        

        final[j][i] = weights[0] * preds_df_fin_xgb.iloc[j:j+1, i].values[0] \
                            + weights[1] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0] \
                        #     + weights[2] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0]

C:\Users\chlje\AppData\Local\Temp\ipykernel_59232\3629398345.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  1 / smapes_df_fin_lgbm.iloc[j:j+1, i].values[0]]
C:\Users\chlje\AppData\Local\Temp\ipykernel_59232\3629398345.py:10: RuntimeWarning: invalid value encountered in true_divide
  weights /= np.sum(weights)
C:\Users\chlje\AppData\Local\Temp\ipykernel_59232\3629398345.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  weights = [1 / smapes_df_fin_xgb.iloc[j:j+1, i].values[0],


In [37]:
final_df = pd.DataFrame(final)
final_values = pd.DataFrame((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0])
final_values_sharpe = -pd.DataFrame(((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0]) / final_df.pct_change().std())

In [38]:
final_values_sharpe.index = dacon_sid_list
final_values_sharpe.columns = ['VALUE']
final_values_sharpe.reset_index(inplace = True)
final_values_sharpe.columns = ['종목코드', 'VALUE']

In [39]:
final_values_sharpe.set_index('종목코드', inplace = True)

In [40]:
final_values_sharpe['VALUE']

종목코드
060310   -2.78023
095570   -2.69839
006840    1.87109
054620   -4.28813
265520   -1.83038
           ...   
189980    -1.7422
000540    1.72483
003280   -8.68666
037440   -1.73952
238490   0.735532
Name: VALUE, Length: 2000, dtype: float64

## Submission

In [ ]:
alpha_feat_insoo = subutil.Submission(
    alpha_series=final_values_sharpe['VALUE'],
    alpha_name='alpha_feat_insoo_lagged_ReverseSharpe-final',
)

In [ ]:
alpha_feat_insoo.get_rank(export_path=OUTPUT_PATH)

# Alpha Strategy by Woohyuk

- 단기, 중기, 중장기에 대한 이동평균선 (3일, 20일, 60일 MA) 을 이용하여 각 주식의 이격도를 구하고, 그것을 New Feature로 이용함. 

- 스케일링된 피쳐들을 사용하여 XGBoost, LightGBM 모델을 훈련시킴. 그런 다음, 모델을 사용하여 검증 세트에 대한 예측을 생성하고, 이 예측 값과 실제 값 사이의 smape를 계산함. smape의 역수에 비례하는 weight를 주어 두 모델을 Ensemble함.

## Basic Settings

### Import libraries

In [47]:
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor

from tqdm import tqdm

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [48]:
## custom library

import eda_util as eutil
import submission_config as subconfig
import submission_util as subutil

In [49]:
pd.set_option('display.float_format', lambda x: f'{x:,g}')

In [50]:
BASE_PATH = subconfig.BASE_PATH
DATA_PATH = subconfig.DATA_PATH

OUTPUT_PATH = subconfig.OUTPUT_PATH

In [51]:
krx_df = pd.read_csv(subconfig.krx_df_PATH)

In [52]:
krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']

In [53]:
krx_df['date'] = pd.to_datetime(krx_df['date'], format='%Y%m%d')

In [54]:
krx_df

,date,code,name,volume,open,high,low,close
0,2021-06-01,A060310,3S,166690,2890,2970,2885,2920
1,2021-06-01,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,2021-06-01,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,2021-06-01,A054620,APS,462544,14600,14950,13800,14950
4,2021-06-01,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,2023-05-30,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,2023-05-30,A000540,흥국화재,50218,3250,3255,3195,3215
987997,2023-05-30,A003280,흥아해운,130664,1344,1395,1340,1370
987998,2023-05-30,A037440,희림,141932,9170,9260,9170,9200


In [55]:
return_df = pd.read_pickle(subconfig.return_df_PATH)
close_df = pd.read_pickle(subconfig.adjclose_df_PATH)
open_df = pd.read_pickle(subconfig.adjopen_df_PATH)
high_df = pd.read_pickle(subconfig.adjhigh_df_PATH)
low_df = pd.read_pickle(subconfig.adjlow_df_PATH)

In [56]:
## date list

holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

In [57]:
TRAIN_START = pd.to_datetime(subconfig.TRAIN_START, format='%Y-%m-%d')
SIMOS_END = pd.to_datetime(subconfig.SIMOS_END, format='%Y-%m-%d')

In [58]:
tradingdays = tradingdays[(tradingdays >= TRAIN_START) & (tradingdays <= SIMOS_END)]

In [59]:
dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()]

In [60]:
return_df = return_df.loc[tradingdays, :].dropna(axis='columns', how='all')
return_df = return_df.loc[:, dacon_sid_list]

close_df = close_df.loc[tradingdays, :].dropna(axis='columns', how='all')
close_df = close_df.loc[:, dacon_sid_list]

In [61]:
open_df = open_df.loc[tradingdays, :].dropna(axis='columns', how='all')
open_df = open_df.loc[:, dacon_sid_list]

high_df = high_df.loc[tradingdays, :].dropna(axis='columns', how='all')
high_df = high_df.loc[:, dacon_sid_list]

low_df = low_df.loc[tradingdays, :].dropna(axis='columns', how='all')
low_df = low_df.loc[:, dacon_sid_list]

In [62]:
SIMOS_START = subconfig.SIMOS_START
# simOS_END = subconfig.SIMOS_END

In [63]:
volume_df = pd.read_pickle(subconfig.volume_df_PATH)
dollarvolume_df = pd.read_pickle(subconfig.dollarvolume_df_PATH)
marketcap_df = pd.read_pickle(subconfig.marketcap_df_PATH)
market_cat_df = pd.read_pickle(DATA_PATH / 'market_cat_df_20140101_20230730.pickle')

In [64]:
volume_df = volume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
volume_df = volume_df.loc[:, dacon_sid_list]

dollarvolume_df = dollarvolume_df.loc[tradingdays, :].dropna(axis='columns', how='all')
dollarvolume_df = dollarvolume_df.loc[:, dacon_sid_list]

marketcap_df = marketcap_df.loc[tradingdays, :].dropna(axis='columns', how='all')
marketcap_df = marketcap_df.loc[:, dacon_sid_list]

In [65]:
market_cat_inrange = market_cat_df[market_cat_df['trdDd'].isin(tradingdays)]

In [66]:
KOSPI_sid_list = market_cat_inrange[market_cat_inrange['is_KOSPI'] == True]['ISU_SRT_CD'].unique()
KOSDAQ_sid_list = market_cat_inrange[market_cat_inrange['is_KOSDAQ'] == True]['ISU_SRT_CD'].unique()
KONEX_sid_list = market_cat_inrange[market_cat_inrange['is_KONEX'] == True]['ISU_SRT_CD'].unique()

## 3일 이평선에 대한 이격도 구하기

In [67]:
disparity_3_df = close_df.copy()
# Calculate 60-day moving average for each stock
moving_avg_3_df = close_df.rolling(window=3).mean()

for column in disparity_3_df.columns:
    disparity_3_df[column] = (disparity_3_df[column] / moving_avg_3_df[column])

disparity_3_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,0.997706,0.991908,1.01633,0.992045,1.00571,0.979979,1.00236,1.01344,0.992138,0.995397,...,0.991952,0.990019,1.01545,1.02343,1.00418,0.996255,1.00178,1,1.05716,1.00581
2021-06-04,1.01143,0.977765,0.991256,0.987298,0.996575,0.986081,0.998588,0.985637,1.01046,1.00293,...,1.00647,1.02304,1.04921,0.994646,1.01177,0.984551,1.00213,1,0.989063,1.01585
2021-06-07,1.05,0.97307,0.975701,1.02288,0.996,1.0182,1.00188,1.02218,0.992126,1.00334,...,1.00964,1.00594,1.02311,0.981985,0.986815,0.98045,0.99431,1,0.955414,1.00571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.0011,1.00265,1.00497,0.996283,1.00538,1.00119,0.998369,1.01115,0.994661,0.997089,...,0.983033,1.00331,0.987034,0.995269,0.988089,1.00777,0.98913,0.976392,0.988433,0.983214
2023-06-16,0.988314,1.04,1.01978,0.988412,1.00307,1.01244,0.990984,1.01501,1.00215,1.00631,...,0.997864,1.0099,1.00359,1,1.03178,1.01653,0.99739,0.989481,1.00338,1.02688
2023-06-19,1.01712,1.01202,1.01957,0.985457,0.991545,1.01155,0.98676,0.977368,1.01068,1.0029,...,1.00375,1.01305,0.997382,0.998813,1.0207,1.00218,0.996855,1.01658,1.02156,1


## 20일 이평선에 대한 이격도 구하기

In [68]:
disparity_20_df = close_df.copy()

In [69]:
# Calculate 20-day moving average for each stock
moving_avg_20_df = close_df.rolling(window=20).mean()

In [70]:
for column in disparity_20_df.columns:
    disparity_20_df[column] = (disparity_20_df[column] / moving_avg_20_df[column])

In [71]:
disparity_20_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.10938,1.00438,0.977442,0.985385,1.02685,0.907882,0.989511,1.02794,0.868714,1.00654,...,0.962167,0.990196,0.936606,0.990554,0.977709,1.01612,0.979021,1.01869,0.959939,1.03967
2023-06-16,1.0713,1.06219,1.01753,0.973623,1.02113,0.924246,0.978155,1.04295,0.876747,1.01357,...,0.971313,0.99951,0.950306,0.993296,1.0308,1.02886,0.982115,1.0102,0.969743,1.07159
2023-06-19,1.09535,1.04858,1.03018,0.957648,1.00632,0.947083,0.966272,0.997341,0.899117,1.01201,...,0.974619,1.01297,0.945248,0.989505,1.03438,1.01992,0.979226,1.03587,0.997495,1.03954


## 60일 이평선에 대한 이격도 구하기

In [72]:
disparity_60_df = close_df.copy()

In [73]:
# Calculate 60-day moving average for each stock
moving_avg_60_df = close_df.rolling(window=60).mean()

In [74]:
for column in disparity_60_df.columns:
    disparity_60_df[column] = (disparity_60_df[column] / moving_avg_60_df[column])

In [75]:
disparity_60_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,1.25823,0.955904,0.994222,0.876808,1.03384,0.754351,0.961584,1.03707,0.836739,1.02925,...,0.957669,1.00295,0.869193,0.993888,1.01417,1.09522,0.974503,1.037,0.949769,1.12137
2023-06-16,1.22381,1.01419,1.02858,0.862743,1.03029,0.76657,0.950715,1.05426,0.835089,1.0364,...,0.965667,1.01221,0.880609,0.997078,1.06904,1.11074,0.974751,1.02988,0.95975,1.1642
2023-06-19,1.26082,1.00434,1.03582,0.854277,1.01723,0.78349,0.938644,1.00898,0.846715,1.03469,...,0.967317,1.02609,0.876164,0.993272,1.07251,1.10297,0.970359,1.05776,0.987282,1.13427


## Parameters

In [76]:
PORTFOLIO_DATE = subconfig.PORTFOLIO_DATE

RDVADV_WINDOW = subconfig.WINDOWS['rdvadv'] # 20

## ADV/RDV Signal

In [77]:
adv_df = dollarvolume_df.rolling(RDVADV_WINDOW, ).mean().dropna(axis='rows', how='all')

In [78]:
avg_adv_s = adv_df.mean(axis='columns')
avg_rdv_s = dollarvolume_df.iloc[RDVADV_WINDOW:, :].mean(axis='columns')

In [79]:
avg_rdvadv_s = avg_rdv_s / avg_adv_s

In [80]:
ii_rdvadv_df = dollarvolume_df.iloc[RDVADV_WINDOW:, :] / adv_df

In [81]:
normalized_rdvadv_signal_df = ii_rdvadv_df.divide(avg_rdvadv_s, axis='rows')
normalized_rdvadv_signal_df

ISU_SRT_CD,060310,095570,006840,054620,265520,211270,027410,282330,126600,138930,...,243070,084110,145020,024060,010240,189980,000540,003280,037440,238490
trdDd,,,,,,,,,,,,,,,,,,,,,
2021-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-29,0.415442,0.240279,0.405709,0.726369,0.475821,0.907576,0.4201,1.10324,0.539159,0.755196,...,1.81974,0.798739,0.631614,0.340136,14.3229,0.996211,0.600184,NaN,0.369902,0.630198
2021-06-30,0.314977,0.212304,0.465795,0.931287,0.470661,0.514831,0.648498,1.99712,0.354948,0.501368,...,4.39418,8.5227,0.745765,0.319291,7.08257,0.416926,1.40616,NaN,0.291685,0.618186
2021-07-01,0.399208,0.295054,0.570299,0.366269,0.496024,0.584884,1.26369,3.66506,0.356092,0.701445,...,1.03469,1.22015,0.426687,0.971435,1.91106,0.786424,0.705502,NaN,0.495063,0.955606
2021-07-02,1.02063,0.146543,0.639072,0.410245,0.842014,0.260692,1.97162,1.5627,0.724157,0.519005,...,2.03734,0.881352,0.244086,2.64395,0.377858,0.422299,0.421016,NaN,1.13061,0.666445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,0.292674,0.588688,1.12985,0.124622,0.90052,0.31337,0.973597,1.10088,0.442653,1.00447,...,0.592776,0.71876,0.580354,0.487591,0.570073,0.493214,1.08468,0.789117,0.514062,0.106241
2023-06-16,0.12223,3.09939,1.38265,0.322443,0.710468,0.296359,1.33772,2.43478,0.495513,1.70201,...,0.44873,0.936876,0.771501,0.2652,2.44716,1.00167,0.93063,0.793416,0.350493,0.14148
2023-06-19,0.31029,1.29243,1.28678,0.15679,0.463203,0.425583,1.23098,1.1466,0.351447,0.838827,...,0.849195,1.65108,0.607847,0.452039,2.08872,0.661718,1.23745,0.757414,1.75569,0.0806887


## Train

In [82]:
MODEL_TRAIN_START = pd.to_datetime('2021-09-01', format='%Y-%m-%d')

In [83]:
# Your function to calculate SMAPE
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [84]:
results_df = pd.DataFrame(columns=['code'] + ['return_day_' + str(i) for i in range(1, 16)])

preds_df_fin_xgb = pd.DataFrame()
smapes_df_fin_xgb = pd.DataFrame()

preds_df_fin_lgbm = pd.DataFrame()
smapes_df_fin_lgbm = pd.DataFrame()


In [ ]:
from prophet import Prophet

smapes_fin_xgb = {}  # Initialize as dictionary
smapes_fin_lgbm = {}  # Initialize as dictionary



preds_fin_xgb = {}  # Initialize as dictionary
preds_fin_lgbm = {}  # Initialize as dictionary


# Iterate over each unique stock
for code in tqdm(dacon_sid_list):
    
    # Filter by stock code
    # Note: All prices are adjusted
    # TODO: Add normalized rdvadv signal to the columns

    train_close = pd.DataFrame(
        data={
            'open': open_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'high': high_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'low': low_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'close': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'close_t-1': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(1),
            'close_t-3': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(3),
            'close_t-5': close_df.loc[MODEL_TRAIN_START:SIMOS_START, code].shift(5),
            'dollarvolume': dollarvolume_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'marketcap': marketcap_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'norm_rdvadv': normalized_rdvadv_signal_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'disparity_3' : disparity_3_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'disparity_20' : disparity_20_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
            'disparity_60' : disparity_60_df.loc[MODEL_TRAIN_START:SIMOS_START, code],
        }
    )
    train_close = train_close.iloc[5:, :] # nan 있는 1st row 제거 

    # Store original data for reference
    original_data = train_close.copy()

    smapes_xgb = []  # Initialize empty lists
    smapes_lgbm = []  # Initialize empty lists
    
    

    preds_xgb = []  # Initialize empty lists
    preds_lgbm = []  # Initialize empty lists
    
    
    # For each day from 1 to 15
    for day in range(1, 16):
        # Scale data
        X = train_close[:]
        y = train_close['close']
        
        scaler = MinMaxScaler(feature_range=(-1, 1))
        data_scaled = scaler.fit_transform(X)
        data_scaled2 = y
        
        X_train = data_scaled[:-day]
        y_train = data_scaled2[day:]
        X_test = data_scaled[-day]
        
        X_train = X_train[:int(len(X_train) * 0.9)]
        X_val = X_train[int(len(X_train) * 0.9):]
        y_train = y_train[:int(len(y_train) * 0.9)]
        y_val = y_train[int(len(y_train) * 0.9):] 

        # Train XGBoost
        xgb_model = XGBRegressor()
        xgb_model.fit(X_train, y_train)
        vals_xgb = xgb_model.predict(X_val)
        smapes_xgb.append(smape(y_val, vals_xgb))
        
        preds_xgb.append(xgb_model.predict([data_scaled[-day]]))

        # Train LightGBM
        lgbm_model = LGBMRegressor(verbose = -1)
        lgbm_model.fit(X_train, y_train)
        vals_lgbm = lgbm_model.predict(X_val)
        smapes_lgbm.append(smape(y_val, vals_lgbm))
        
        preds_lgbm.append(lgbm_model.predict([data_scaled[-day]]))
        
    
    smapes_fin_xgb[code] = smapes_xgb
    smapes_fin_lgbm[code] = smapes_lgbm
    

    preds_fin_xgb[code] = preds_xgb
    preds_fin_lgbm[code] = preds_lgbm
    

# Convert the dictionaries to DataFrames after the loop
smapes_df_fin_xgb = pd.DataFrame(smapes_fin_xgb)
smapes_df_fin_lgbm = pd.DataFrame(smapes_fin_lgbm)


preds_df_fin_xgb = pd.DataFrame(preds_fin_xgb)
preds_df_fin_lgbm = pd.DataFrame(preds_fin_lgbm)

In [ ]:
preds_df_fin_lgbm.shape

In [ ]:
final = np.zeros((len(smapes_df_fin_xgb), len(smapes_df_fin_xgb.columns)))

for i in range(0, len(smapes_df_fin_xgb.columns)):
    for j in range(0, len(smapes_df_fin_xgb)):
        weights = [1 / smapes_df_fin_xgb.iloc[j:j+1, i].values[0],
                   
                   1 / smapes_df_fin_lgbm.iloc[j:j+1, i].values[0]]
                  
                   
        

        weights /= np.sum(weights) 
        

        final[j][i] = weights[0] * preds_df_fin_xgb.iloc[j:j+1, i].values[0] \
                            + weights[1] * preds_df_fin_lgbm.iloc[j:j+1, i].values[0] 

In [ ]:
final_df = pd.DataFrame(final)
final_values = pd.DataFrame((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0])
final_values_sharpe = -pd.DataFrame(((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0]) / final_df.pct_change().std())

In [ ]:
final_values

In [ ]:
final_values_sharpe.index = dacon_sid_list
final_values_sharpe.columns = ['VALUE']
final_values_sharpe.reset_index(inplace = True)
final_values_sharpe.columns = ['종목코드', 'VALUE']

In [ ]:
final_values_sharpe.set_index('종목코드', inplace = True)

In [ ]:
final_values_sharpe['VALUE']

In [ ]:
alpha_feat_insoo = subutil.Submission(
    alpha_series=final_values_sharpe['VALUE'],
    alpha_name='alpha_feat_insoo_lagged_ReverseSharpe_disparity_added_2',
)

In [ ]:
alpha_feat_insoo.get_rank(export_path=OUTPUT_PATH)

# Alpha Strategy by Insu

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import pandas_ta as ta
import pycaret
from pycaret.regression import setup, compare_models

## Import Data

- 베이스 파일 내의 `data` 폴더에 `train.csv` , `train_additional.csv` 파일이 첨부되어 있는지 확인!

In [96]:
train = pd.read_csv('./data/train.csv')
train_2 = pd.read_csv('./data/train_additional.csv')
train = pd.concat([train, train_2], axis = 0)
train.reset_index(drop = True)

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
1071995,20230728,A001080,만호제강,12964,35550,36000,34700,36000
1071996,20230728,A104700,한국철강,72644,5780,6030,5780,6030
1071997,20230728,A045100,한양이엔지,59562,16230,16390,15970,16330
1071998,20230728,A000020,동화약품,86169,9870,10080,9700,9800


In [97]:
# Your function to calculate SMAPE
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [98]:
# Initialize results DataFrame
results_df = pd.DataFrame(np.random.rand(15, 2000))
results_df.columns = train['종목코드'].unique()

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

preds_df_fin = pd.DataFrame()

In [99]:
results_df.index = ['2023-07-31', '2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
                    
'2023-08-07', '2023-08-08', '2023-08-09', '2023-08-11', '2023-08-14',
 '2023-08-16', '2023-08-17', '2023-08-18', '2023-08-21', '2023-08-22']

## Train

In [ ]:
# Iterate over each unique stock
for code in tqdm(unique_codes):

    df = train[train['종목코드'] == code][['일자', '시가', '고가', '저가',  '종가', '거래량']]
    df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
    df.set_index('일자', inplace=True)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    for sma_n in range(1, 5):
        df[f'SMA_{sma_n}'] = df.ta.sma(close='Close', length=sma_n)

    for ema_n in range(1, 5):
        df[f'EMA_{ema_n}'] = df.ta.ema(close='Close', length=ema_n)

    df['RSI_14'] = df.ta.rsi(close='Close', length=14)  # Relative Strength Index
    df['MACD'] = df.ta.macd(close='Close')['MACD_12_26_9']  # Moving Average Convergence Divergence
    df['ADX'] = df.ta.adx(high='High', low='Low', close='Close')['ADX_14']  # Average Directional Index
    df['ATR_14'] = df.ta.atr(high='High', low='Low', close='Close')  # Average True Range
    df['CCI_14'] = df.ta.cci(high='High', low='Low', close='Close')  # Commodity Channel Index
    df['ROC_10'] = df.ta.roc(close='Close')  # Rate of Change
    # For each day from 1 to 15
    values = []
    
    for day in tqdm(range(1, 16)):
        # Scale data
        X = df
        y = df['Close']

        scaler = MinMaxScaler(feature_range=(0, 1))
        data_scaled = scaler.fit_transform(X)
        data_scaled2 = y

        X_train = data_scaled[:-day]
        y_train = data_scaled2[day:]
        X_test = data_scaled[-day]

        X_train = pd.DataFrame(X_train)
        y_train = pd.DataFrame(y_train)
        X_test = pd.DataFrame(X_test).T
        
        X_train.columns = X.columns
        y_train.columns = ['Target']
        X_test.columns = X.columns
        
        X_train.index = y_train.index
        X_test.index = [X.index[-day]]
        
        train_set = pd.concat([X_train, y_train], axis = 1).dropna()
        reg = setup(data = train_set, target = 'Target')
    # compare all models and select top 5
        model = compare_models(n_select = 1, cross_validation = False, verbose = False)
        value = model.predict(X_test)
        values.append(value)
        
    results_df[code] = values

In [ ]:
final_df = pd.DataFrame(final)
final_values = pd.DataFrame((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0])
final_values_sharpe = -pd.DataFrame(((final_df.iloc[-1] - final_df.iloc[0]) / final_df.iloc[0]) / final_df.pct_change().std())

In [ ]:
final_values_sharpe

In [ ]:
final_values_sharpe.index = unique_codes
final_values_sharpe.columns = ['VALUE']
final_values_sharpe.reset_index(inplace = True)
final_values_sharpe.columns = ['종목코드', 'VALUE']

In [ ]:
# Submit
# Set the ranks for NaNs starting from 1800 and decreasing
nan_indices = np.setdiff1d(unique_codes, final_values_sharpe['종목코드'].values)

# Drop the rows with NaNs from the original DataFrame
baseline_submission = final_values_sharpe[~final_values_sharpe['종목코드'].isin(nan_indices)]
final_values_sharpe.sort_values('VALUE', ascending = False, inplace = True)
baseline_submission.sort_values('VALUE', ascending = False, inplace = True)

# Split the remaining rank range into two parts: before and after the ranks assigned to NaNs
before_nan_ranks = np.arange(1, 1801 - len(nan_indices))
after_nan_ranks = np.arange(1801, 2001)
nan_ranks = np.arange(1801 - len(nan_indices), 1801)

# Assign the remaining ranks
baseline_submission['순위'] = np.concatenate([before_nan_ranks, after_nan_ranks])
nan_vals = pd.DataFrame()
nan_vals['종목코드'] = nan_indices
nan_vals['VALUE'] = np.nan
nan_vals['순위'] = nan_ranks

# Insert the rows with NaNs (now ranked)
baseline_submission = pd.concat([baseline_submission, nan_vals])

# Sort the DataFrame by '순위'
baseline_submission = baseline_submission.sort_values(by='순위')
# Ensure '순위' is of integer type
baseline_submission['순위'] = baseline_submission['순위'].astype('int')
baseline_submission = baseline_submission.drop(['VALUE'], axis = 1)
sample_submission = pd.read_csv('./sample_submission.csv')
baseline_submission = sample_submission[['종목코드']].merge(baseline_submission[['종목코드', '순위']], on='종목코드', how='left')

In [ ]:
baseline_submission.to_csv('./output/CIS.csv', index=False)

## Submission Ensemble (Voting System)

In [ ]:
import pandas as pd
import numpy as np

import submission_util as subutil
import submission_config as subconfig

df1 = pd.read_csv(subconfig.OUTPUT_PATH / 'alpha_feat_insoo_lagged_ReverseSharpe-final.csv', index_col = 0)
df2 = pd.read_csv(subconfig.OUTPUT_PATH / 'CIS.csv', index_col = 0)
df3 = pd.read_csv(subconfig.OUTPUT_PATH / 'alpha_feat_insoo_lagged_ReverseSharpe_disparity_added_2.csv', index_col = 0)

In [ ]:
df1.sort_index(inplace = True)
df2.sort_index(inplace = True)
df3.sort_index(inplace = True)

In [ ]:
def rank_to_vote(rank):
    if rank <= 200:
        return 1
    elif rank >= 1801:
        return -1
    else:
        return 0

In [ ]:
df1['vote'] = df1['순위'].apply(lambda x: rank_to_vote(x))
df2['vote'] = df2['순위'].apply(lambda x: rank_to_vote(x))
df3['vote'] = df3['순위'].apply(lambda x: rank_to_vote(x))

In [ ]:
final = df1['vote'] + df2['vote'] + df3['vote']
final.index = [code[1:] for code in final.index]
final

## Submission

In [ ]:
final_submission = subutil.Submission(final, 'final')

In [ ]:
final_submission.get_rank(export_path=subconfig.OUTPUT_PATH)

- 이렇게 하면, output 폴더에 `final.csv` 파일이 생성됩니다.
- 이 `final.csv` 파일이 곧 '정답 파일' 입니다.

## 함께 사용된 Python 모듈

`submission_config.py`

In [ ]:
from pathlib import Path

## Path configs

BASE_PATH = Path('.').resolve()
DATA_PATH = BASE_PATH / 'data'
OUTPUT_PATH = BASE_PATH / 'output'

krx_df_PATH = DATA_PATH / 'train.csv'
return_df_PATH = DATA_PATH / 'return_20140101_20230730.pkl'
adjclose_df_PATH = DATA_PATH / 'adjClose_20140101_20230730.pkl'
adjhigh_df_PATH = DATA_PATH / 'adjHigh_20140101_20230730.pkl'
adjlow_df_PATH = DATA_PATH / 'adjLow_20140101_20230730.pkl'
adjopen_df_PATH = DATA_PATH / 'adjOpen_20140101_20230730.pkl'
volume_df_PATH = DATA_PATH / 'volume_df_20140101_20230730.pkl'
dollarvolume_df_PATH = DATA_PATH / 'dollarvolume_df_20140101_20230730.pkl'
marketcap_df_PATH = DATA_PATH / 'marketcap_df_20140101_20230730.pkl'

## Param configs

# train (custom)
TRAIN_START = '2021-06-01'

# SimOS
PORTFOLIO_DATE = '2023-05-30' 
SIMOS_START = '2023-05-31'
SIMOS_END = '2023-06-21'

# RealOS
REALOS_PORTFOLIO_DATE = '2023-07-28' 
REALOS_START = '2023-07-31'

WINDOWS = {
    'rdvadv': 20,
}

`submission_util.py`

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    roc_curve, 
    auc
    )
# TODO: SimOS 에서의 정답을 알고있다. 그러므로 eval metric 계산할 수 있다. 

import submission_config as subconfig

## Params
DACON_SID_CNT = 2000
SIMOS_START = subconfig.SIMOS_START
SIMOS_END = subconfig.SIMOS_END

## Import data
krx_df = pd.read_csv(subconfig.krx_df_PATH)
adjclose_df = pd.read_pickle(subconfig.adjclose_df_PATH)
return_df = pd.read_pickle(subconfig.return_df_PATH)

def get_simos_data(return_df, adjclose_df):
    holidays = return_df.isnull().all(axis=1)
    tradingdays = ~holidays

    holidays = holidays.index[holidays]
    tradingdays = tradingdays.index[tradingdays]

    return_df = return_df.loc[tradingdays, :]
    adjclose_df = adjclose_df.loc[tradingdays, :]

    return_df = return_df.loc[SIMOS_START:SIMOS_END, :]
    adjclose_df = adjclose_df.loc[SIMOS_START:SIMOS_END, :]

    return return_df, adjclose_df

# TODO: Confusing if global variables are not capitalized
simos_return_df, simos_adjclose_df = get_simos_data(return_df, adjclose_df) # simos period, only trading days

## for filtering
def get_tradables(adjclose_df, trading_date=subconfig.PORTFOLIO_DATE):
    sid_list = adjclose_df.columns

    notnull = adjclose_df.loc[trading_date, :].notnull()
    notzero = adjclose_df.loc[trading_date, :] != 0

    return sid_list[notnull * notzero]

def is_tradables(sid_list, tradables):
    tradables = set(tradables)

    return np.array([True if sid in tradables else False for sid in sid_list])

def get_daconsids(krx_df):
    krx_df.columns = ['date', 'code', 'name', 'volume', 'open', 'high', 'low', 'close']
    dacon_sid_list = [ii[1:] for ii in krx_df['code'].unique()] # 060310 형식으로 바꿔줌

    return dacon_sid_list

def is_daconsids(sid_list, daconsids):
    daconsids = set(daconsids)

    return np.array([True if sid in daconsids else False for sid in sid_list])

class Submission:
    holding_return_s = (simos_adjclose_df.loc[SIMOS_END, :] - simos_adjclose_df.loc[SIMOS_START, :]).divide(simos_adjclose_df.loc[SIMOS_START, :])  
    holding_return_s = holding_return_s.fillna(0)

    # simos_winners = 
    # TODO: Add data science evaluation metrics

    # TODO: Make not-instance-specific variables to class variables
    def __init__(self, alpha_series:pd.Series, alpha_name:str, top=200, bottom=200):
        self.alpha_series = alpha_series
        self.alpha_name = alpha_name
        self.top = top
        self.bottom = bottom

        self.sid_list = self.alpha_series.index
        self.tradables = get_tradables(adjclose_df)
        self.daconsids = get_daconsids(krx_df)
    
        self.is_selectables = is_tradables(self.sid_list, self.tradables) * is_daconsids(self.sid_list, self.daconsids)
        self.submission_df = None
        self.alpha_winners = None
        self.alpha_losers = None

        # for excess return
        self.long_hpr = None
        self.short_hpr = None
        self.final_return = None

        # for variance
        self.long_returns = None
        self.short_returns = None
        
    def get_rank(self, export_path=None):
        selectables = self.alpha_series[self.is_selectables]
        top_s = selectables.nlargest(self.top)
        bottom_s = selectables.nsmallest(self.bottom)
        
        self.alpha_winners = top_s.index
        self.alpha_losers = bottom_s.index
        
        submission_df = pd.DataFrame(
            data={'rank': [-1]*DACON_SID_CNT},
            index=self.daconsids
        )
        submission_df.index.name = 'sid'

        submission_df['rank'][top_s.index] = np.arange(1, self.top+1)
        submission_df['rank'][bottom_s.index] = np.arange(DACON_SID_CNT, DACON_SID_CNT - self.bottom, -1)

        submission_df['rank'][submission_df['rank'] == -1] = np.arange(self.top+1, DACON_SID_CNT - self.bottom + 1)

        self.submission_df = submission_df

        if export_path:
            submission_df.index = ['A' + idx for idx in submission_df.index]
            submission_df.index.name = '종목코드'
            submission_df.columns = ['순위']
            submission_df.to_csv(export_path / f'{self.alpha_name}.csv', encoding='utf-8')
            
            print(f'Saved to {export_path / self.alpha_name}.csv')
            return submission_df

        return submission_df

    def get_excess_return(self, risk_free_rate=0.035, days_of_trading=15):
        self.long_hpr = Submission.holding_return_s[self.alpha_winners].sum()
        self.short_hpr = Submission.holding_return_s[self.alpha_losers].sum()

        self.final_return = (self.long_hpr - self.short_hpr) / 400

        annualized_final_return = self.final_return * 250 / days_of_trading
        excess_return = annualized_final_return - risk_free_rate

        return excess_return
    
    def get_volatility(self, days_of_trading=15):
        self.long_returns = simos_return_df.loc[:, self.alpha_winners].mean(axis=1)
        self.short_returns = simos_return_df.loc[:, self.alpha_losers].mean(axis=1)

        annualized_portfolio_returns = (self.long_returns - self.short_returns) / 2 * 250
        annualized_mean_returns = annualized_portfolio_returns.mean()
        
        annualized_portfolio_volatility = np.sqrt((annualized_portfolio_returns - annualized_mean_returns).pow(2)[2:].sum() / (days_of_trading-2))

        return annualized_portfolio_volatility

    def get_Sharpe(self):
        return self.get_excess_return() / self.get_volatility()

    
class Score:
    holding_return_s = (simos_adjclose_df.loc[SIMOS_END, :] - simos_adjclose_df.loc[SIMOS_START, :]).divide(simos_adjclose_df.loc[SIMOS_START, :])  
    holding_return_s = holding_return_s.fillna(0)

    def __init__(self, submission_csv_filepath, alpha_name, top=200, bottom=200, encoding='utf-8'):
        self.alpha_name = alpha_name
        self.top = top
        self.bottom = bottom

        with open(submission_csv_filepath, 'r', encoding=encoding) as f:
            submission_df = pd.read_csv(f, index_col=0)
        
        submission_df.index = [idx[1:] for idx in submission_df.index]
        submission_df.index.name = 'sid'
        submission_df.columns = ['rank']

        self.alpha_series = submission_df['rank']
        self.sid_list = self.alpha_series.index

        # TODO: Add validations

        self.submission_df = None
        self.alpha_winners = self.alpha_series.nsmallest(self.top).index
        self.alpha_losers = self.alpha_series.nlargest(self.bottom).index

        # for excess return
        self.long_hpr = None
        self.short_hpr = None
        self.final_return = None

        # for variance
        self.long_returns = None
        self.short_returns = None
    
    def get_excess_return(self, risk_free_rate=0.035, days_of_trading=15):
        self.long_hpr = Score.holding_return_s[self.alpha_winners].sum()
        self.short_hpr = Score.holding_return_s[self.alpha_losers].sum()

        self.final_return = (self.long_hpr - self.short_hpr) / 400

        annualized_final_return = self.final_return * 250 / days_of_trading
        excess_return = annualized_final_return - risk_free_rate

        return excess_return

    def get_volatility(self, days_of_trading=15):
        self.long_returns = simos_return_df.loc[:, self.alpha_winners].mean(axis=1)
        self.short_returns = simos_return_df.loc[:, self.alpha_losers].mean(axis=1)

        annualized_portfolio_returns = (self.long_returns - self.short_returns) / 2 * 250
        annualized_mean_returns = annualized_portfolio_returns.mean()
        
        annualized_portfolio_volatility = np.sqrt((annualized_portfolio_returns - annualized_mean_returns).pow(2)[2:].sum() / (days_of_trading-2))

        return annualized_portfolio_volatility

    def get_Sharpe(self):
        sharpe = self.get_excess_return() / self.get_volatility()
        print(f'Sharpe of {self.alpha_name}: {sharpe}')

        return sharpe